In [ ]:
import pandas
import numpy
import pymysql
import datetime
import pytz
import time
from youtrack.connection import Connection

print("Press 1 for davidd \n", "Press 2 for gabriel \n", "Press 3 for leela \n", "Press 4 for mikew \n", "Press 5 for william.alvarez \n \n \n CHOICE:  ")
user_in = str(input())
buick_user = ''
if user_in == '1':
    buick_user = 'user1'
elif user_in == '2':
    buick_user = 'user2'
elif user_in == '3':
    buick_user = 'user3'
elif user_in == '4':
    buick_user = 'user4'
elif user_in == '5':
    buick_user = 'user5'
print(buick_user)


print("\n \n \n Please enter Reservation No: ")
res_no = str(input())

print("\n \n \n Please enter Owner's ZD #: ")
o_zd = str(input())

print("\n \n \n Please enter Renter's ZD #: ")
r_zd = str(input())

#res_no = '1682838'

#########################################################################################################
# GET DATA


#Open MySQL connection
conn = pymysql.connect(host='host',user='user',password='password', connect_timeout=1000, cursorclass=pymysql.cursors.DictCursor)
a=conn.cursor()

#analytics.reservation_summary table
sql= 'SELECT * FROM analytics.reservation_summary WHERE reservation_id=' + res_no
a.execute(sql)
res_sum = pandas.read_sql_query(sql, conn)
res_id = pandas.DataFrame.get_value(res_sum, index=0, col='reservation_id', takeable=False)
loc_id = pandas.DataFrame.get_value(res_sum, index=0, col='location_id', takeable=False)
res_start = pandas.DataFrame.get_value(res_sum, index=0, col='location_id', takeable=False)
res_end = pandas.DataFrame.get_value(res_sum, index=0, col='location_id', takeable=False)
vehicle_id = pandas.DataFrame.get_value(res_sum, index=0, col='vehicle_id', takeable=False)
driver_id = pandas.DataFrame.get_value(res_sum, index=0, col='driver_id', takeable=False)
owner_id = pandas.DataFrame.get_value(res_sum, index=0, col='owner_id', takeable=False)

#relayrides.location table 
sql = 'SELECT *, CONCAT(street_number, " ", street_name, ", ", city, ", ", state, ", ", zip) AS pickup_addr FROM relayrides.location WHERE id=' + str(loc_id)
res_loc = pandas.read_sql_query(sql, conn)
pickup_addr = pandas.DataFrame.get_value(res_loc, index=0, col='pickup_addr', takeable=False)

#relayrides.reservation table
sql= 'SELECT * FROM relayrides.reservation WHERE id=' + res_no
a.execute(sql)
reservation = pandas.read_sql_query(sql, conn)
res_start = pandas.DataFrame.get_value(reservation, index=0, col='trip_start', takeable=False)
res_end = pandas.DataFrame.get_value(reservation, index=0, col='trip_end', takeable=False)

#analytics.vehicle_summary table
sql= 'SELECT *, CONCAT(year, " ", make, " ", model) AS v_make_model FROM analytics.vehicle_summary WHERE vehicle_id=' + str(vehicle_id)
a.execute(sql)
vehicle_summary = pandas.read_sql_query(sql, conn)
v_make_model = pandas.DataFrame.get_value(vehicle_summary, index=0, col='v_make_model', takeable=False)
tmv = pandas.DataFrame.get_value(vehicle_summary, index=0, col='tmv_amt', takeable=False)

#relayrides.vehicle table
sql= 'SELECT * FROM relayrides.vehicle WHERE id=' + str(vehicle_id)
a.execute(sql)
vehicle = pandas.read_sql_query(sql, conn)
v_plate = pandas.DataFrame.get_value(vehicle, index=0, col='license_plate_number', takeable=False)
v_state = pandas.DataFrame.get_value(vehicle, index=0, col='license_plate_state', takeable=False)
v_name = pandas.DataFrame.get_value(vehicle, index=0, col='name', takeable=False)
v_status = pandas.DataFrame.get_value(vehicle, index=0, col='vehicle_status', takeable=False)
v_color = pandas.DataFrame.get_value(vehicle, index=0, col='color', takeable=False)

#analytics.driver_summary table - RENTER
sql= 'SELECT *, CONCAT(first_name, " ", last_name) AS name FROM analytics.driver_summary WHERE driver_id=' + str(driver_id)
a.execute(sql)
renter = pandas.read_sql_query(sql, conn)
r_name = pandas.DataFrame.get_value(renter, index=0, col='name', takeable=False)
r_email = pandas.DataFrame.get_value(renter, index=0, col='email', takeable=False)

#analytics.driver_summary table - OWNER
sql= 'SELECT *, CONCAT(first_name, " ", last_name) AS name FROM analytics.driver_summary WHERE driver_id=' + str(owner_id)
a.execute(sql)
owner = pandas.read_sql_query(sql, conn)
o_name = pandas.DataFrame.get_value(owner, index=0, col='name', takeable=False)
o_mail = pandas.DataFrame.get_value(owner, index=0, col='email', takeable=False)

#relayrides.mobile_phone table - RENTER
sql= 'SELECT * FROM relayrides.mobile_phone WHERE driver_id=' + str(driver_id)
a.execute(sql)
renter_phone = pandas.read_sql_query(sql, conn)
r_phone = pandas.DataFrame.get_value(renter_phone, index=0, col='phone_number', takeable=False)

#relayrides.mobile_phone table - OWNER
sql= 'SELECT * FROM relayrides.mobile_phone WHERE driver_id=' + str(owner_id)
a.execute(sql)
owner_phone = pandas.read_sql_query(sql, conn)
o_phone = pandas.DataFrame.get_value(owner_phone, index=0, col='phone_number', takeable=False)

#relayrides.contact_information table - RENTER
sql= 'SELECT *, CONCAT(street_address, ", ", city, ", ", state, " ", zip) AS address FROM relayrides.contact_information WHERE driver_id=' + str(driver_id) + ' ORDER BY id DESC LIMIT 1'
a.execute(sql)
renter_contact = pandas.read_sql_query(sql, conn)
r_addr = pandas.DataFrame.get_value(renter_contact, index=0, col='address', takeable=False)

#relayrides.contact_information table - OWNER
sql= 'SELECT *, CONCAT(street_address, ", ", city, ", ", state, " ", zip) AS address FROM relayrides.contact_information WHERE driver_id=' + str(owner_id) + ' ORDER BY id DESC LIMIT 1'
a.execute(sql)
owner_contact = pandas.read_sql_query(sql, conn)
o_addr = pandas.DataFrame.get_value(owner_contact, index=0, col='address', takeable=False)

#CLOSE MYSQL CONNECTION
conn.close()


#########################################################################

#New Buick Ticket
desc = str(v_make_model) + " | R: " + str(r_name) + " | O: " + str(o_name) + "\n \n"

#summary = 'test'
summary = "*Renter Communication (ZD):* https://relayrides.zendesk.com/agent/tickets/" + str(r_zd) +"\n" + "*Owner Communication (ZD):* https://relayrides.zendesk.com/agent/tickets/" + str(o_zd) + "\n \n" + "*Summary:* \n" + "+====Reservation Information====+ \n" + "*Reservation ID:*  https://turo.com/admin/reservation/edit?reservationId=" + str(res_id) + "\n" + "*Region:* \n" + "*Vehicle Exchange Address:* " + str(pickup_addr) + "\n" + "*Trip Start Date:* " + str(res_start) + "\n" + "*Trip End Date:* " + str(res_end) + "\n \n" + "+====Vehicle Information====+ \n" + "*Vehicle Status:* " + str(v_status) + "\n" + "*Year/Make/Model:* " + str(v_make_model) + "\n" + "*Vehicle Color:* " + str(v_color) + "\n" + "*Plate #:* " + str(v_plate) + "\n" + "*State:* " + str(v_state) + "\n" + "*VIN #:* \n" + "*TMV:* " + str(tmv) + "\n"+ "*Description:* " + str(v_name) + "\n" + "*Listing URL:* \n \n" + "+====Parties Involved====+ \n" + "+*RENTER/SUSPECT*+ \n"+ "*Name:* " + str(r_name) + "\n" + "*Ph# Verified:* " + str(r_phone) + "\n" + "*Home Address:* " + str(r_addr) + "\n" + "*Billing Address:* \n" + "*Email Verified:* " + str(r_email) + "\n" + "*Notes:* \n \n" + "----- \n \n" + "+*OWNER/VICTIM/CUSTODIAN*+ \n" + "*Name:* " + str(o_name) + "\n" + "*Ph # Verified:* " + str(o_phone) + "\n" + "*Home Address:* " + str(o_addr) + "\n" + "*Notes:* \n \n" + "-----"



#########################################################################

# CREATE BUICK TICKET

connection = Connection('https://0000000.com', 'user', 'password')

connection.create_issue('TS', str(buick_user), str(desc), str(summary), 'Normal', 'Missing', 'Escalated')